<a href="https://colab.research.google.com/github/sheltonwlsn/image_classifier_pytorch/blob/main/image_classifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.cuda import is_available

In [2]:
# Check if CUDA is available
if is_available():
    print("CUDA is available")

train = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

dataset = DataLoader(train, batch_size=10)

device = "cuda" if is_available() else "cpu"

CUDA is available


100%|██████████| 9.91M/9.91M [00:10<00:00, 911kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 246kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.64MB/s]


In [4]:
class ImageClassifier(nn.Module):

  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
      nn.Conv2d(1, 32, (3,3)),
      nn.ReLU(),
      nn.Conv2d(32, 64, (3,3)),
      nn.ReLU(),
      nn.Conv2d(64, 64, (3,3)),
      nn.ReLU(),
      nn.Flatten(),
      nn.Linear(64*(28-6)*(28-6), 10)
    )

  def forward(self, x):
    x = self.model(x)
    return x

model = ImageClassifier().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

if __name__ == "__main__":
  epochs = 10
  for epoch in range(epochs):
    for batch in dataset:
      X, y = batch
      X, y = X.to(device), y.to(device)
      pred = model(X)
      loss = loss_fn(pred, y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


# Save the model
with open("model.pt", "wb") as f:
  save(model.state_dict(), f)

Streaming output truncated to the last 5000 lines.
Epoch 10/10, Loss: 1.6689287463123037e-07
Epoch 10/10, Loss: 8.344646573732462e-08
Epoch 10/10, Loss: 7.033335691630782e-07
Epoch 10/10, Loss: 0.0
Epoch 10/10, Loss: 2.8609820219571702e-06
Epoch 10/10, Loss: 0.0
Epoch 10/10, Loss: 3.4570635421005136e-07
Epoch 10/10, Loss: 1.4208737411536276e-05
Epoch 10/10, Loss: 2.3007132767816074e-06
Epoch 10/10, Loss: 0.0021007531322538853
Epoch 10/10, Loss: 0.0
Epoch 10/10, Loss: 2.2411095415009186e-06
Epoch 10/10, Loss: 1.192092824453539e-08
Epoch 10/10, Loss: 3.576278118089249e-08
Epoch 10/10, Loss: 2.741809908002324e-07
Epoch 10/10, Loss: 2.384185471271394e-08
Epoch 10/10, Loss: 1.192092824453539e-08
Epoch 10/10, Loss: 1.192092824453539e-08
Epoch 10/10, Loss: 5.006777428206988e-07
Epoch 10/10, Loss: 2.9802276912960224e-07
Epoch 10/10, Loss: 6.174850568640977e-06
Epoch 10/10, Loss: 0.00043522875057533383
Epoch 10/10, Loss: 0.03605322167277336
Epoch 10/10, Loss: 4.768366181906458e-07
Epoch 10/10, 

In [7]:
import torch
from PIL import Image


class ImageClassifierPredictionModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
      nn.Conv2d(1, 32, (3,3)),
      nn.ReLU(),
      nn.Conv2d(32, 64, (3,3)),
      nn.ReLU(),
      nn.Conv2d(64, 64, (3,3)),
      nn.ReLU(),
      nn.Flatten(),
      nn.Linear(64*(28-6)*(28-6), 10)
    )

  def forward(self, x):
    x = self.model(x)
    return x

model = ImageClassifierPredictionModel().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

if __name__ == "__main__":
  with open("model.pt", "rb") as f:
    model.load_state_dict(load(f))

  img_1 = Image.open('img_1.jpg')
  img_tensor_1 = ToTensor()(img_1).unsqueeze(0).to(device)
  img_2 = Image.open('img_2.jpg')
  img_tensor_2 = ToTensor()(img_2).unsqueeze(0).to(device)
  img_3 = Image.open('img_2.jpg')
  img_tensor_3 = ToTensor()(img_3).unsqueeze(0).to(device)

  print(torch.argmax(model(img_tensor_1)))
  print(torch.argmax(model(img_tensor_2)))
  print(torch.argmax(model(img_tensor_3)))


tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
